<a href="https://colab.research.google.com/github/andrewdk1123/KoSentiment/blob/main/LSTMSentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

In [5]:
# Load packages
import pandas as pd
import io

## Upload Train and Test Set

In [2]:
from google.colab import files

uploaded = files.upload()

Saving processed_training.csv to processed_training.csv


In [3]:
print(uploaded.keys())

dict_keys(['processed_training.csv'])


In [8]:
train_data = pd.read_csv(io.BytesIO(uploaded['processed_training.csv']), sep = ',')

cols_to_keep = ['label', 'sentence', 'tokenized_sentence', 'cleaned_tokens']
train_data = train_data.loc[:, cols_to_keep]
train_data.head()

,label,sentence,tokenized_sentence,cleaned_tokens
0,0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,"['일은', '왜', '해도', '해도', '끝이', '없을까', '?', '화가'...","['일은', '왜', '해도', '해도', '끝이', '없을까', '화가', '난다']"
1,0,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,"['이번', '달에', '또', '급여', '##가', '깎', '##였어', '!...","['이번', '달에', '또', '급여', '깎', '물가', '오르는', '월급'..."
2,0,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,"['회사에', '신입', '##이', '들어왔', '##는데', '말투', '##가...","['회사에', '신입', '들어왔', '말투', '거슬', '그런', '애를', '..."
3,0,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,"['직장', '##에서', '막내', '##라는', '이유로', '나에게', '##...","['직장', '막내', '이유로', '나에게', '온갖', '심', '시켜', '일..."
4,0,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,"['얼마', '전', '입사', '##한', '신입', '##사원', '##이', ...","['얼마', '전', '입사', '신입', '나를', '무시', '것', '같아서'..."


In [9]:
uploaded = files.upload()

Saving processed_test.csv to processed_test.csv


In [10]:
print(uploaded.keys())

dict_keys(['processed_test.csv'])


In [13]:
test_data = pd.read_csv(io.BytesIO(uploaded['processed_test.csv']), sep = ',')

test_data = test_data.loc[:, cols_to_keep]
test_data.head()

,label,sentence,tokenized_sentence,cleaned_tokens
0,0,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...,"['이번', '프로젝트', '##에서', '발표를', '하는데', '내가', '실수...","['이번', '프로젝트', '발표를', '하는데', '내가', '실수', '바람에'..."
1,0,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.,"['회사에서', '중요한', '프로젝트를', '혼자', '하게', '됐는데', '솔...","['회사에서', '중요한', '프로젝트를', '혼자', '하게', '됐는데', '솔..."
2,0,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.,"['상', '##사가', '너무', '무섭게', '생겨서', '친', '##해지는'...","['상', '너무', '무섭게', '생겨서', '친', '게', '너무', '두려워']"
3,0,이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.,"['이번에', '힘들게', '들어간', '첫', '직장', '##이거', '##든'...","['이번에', '힘들게', '들어간', '첫', '직장', '첫', '직장', '그..."
4,0,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼.,"['직장', '##에서', '동료', '##들이랑', '관계가', '안', '좋아질...","['직장', '동료', '관계가', '안', '좋아질', '봐', '걱정']"


##

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load and apply KoBERT tokenizer to sentence
tokenizer = BertTokenizer.from_pretrained('kykim/bert-kor-base')

In [15]:
# Build a vocabulary mapping
vocab = {word: idx + 1 for idx, (word, _) in enumerate(tokenizer.get_vocab().items())}

# Create reverse_vocab by swapping keys and values
reverse_vocab = {idx: word for word, idx in vocab.items()}

# Convert cleaned tokens to numerical indices
train_data['numerical_tokens'] = train_data['cleaned_tokens'].apply(lambda tokens: [vocab.get(token, 0) for token in tokens])

# Pad sequences using PyTorch
padded_sequences = pad_sequence([torch.tensor(tokens) for tokens in train_data['numerical_tokens']], batch_first=True, padding_value=0)

# Add the padded_sequences to the DataFrame
train_data['padded_sequences'] = [list(seq) for seq in padded_sequences]

# Display the DataFrame
print(train_data['padded_sequences'])

0        [tensor(2036), tensor(0), tensor(5930), tensor...
1        [tensor(2036), tensor(0), tensor(5922), tensor...
2        [tensor(2036), tensor(0), tensor(7852), tensor...
3        [tensor(2036), tensor(0), tensor(6268), tensor...
4        [tensor(2036), tensor(0), tensor(5599), tensor...
                               ...                        
57749    [tensor(2036), tensor(0), tensor(2431), tensor...
57750    [tensor(2036), tensor(0), tensor(2431), tensor...
57751    [tensor(2036), tensor(0), tensor(2431), tensor...
57752    [tensor(2036), tensor(0), tensor(2784), tensor...
57753    [tensor(2036), tensor(0), tensor(2431), tensor...
Name: padded_sequences, Length: 57754, dtype: object


In [16]:
print("Max Sequence Length:", max(len(tokens) for tokens in train_data['cleaned_tokens']))
print(len(train_data['padded_sequences'][0]) == max(len(tokens) for tokens in train_data['cleaned_tokens']))

print(train_data['padded_sequences'][0])

Max Sequence Length: 240
True
[tensor(2036), tensor(0), tensor(5930), tensor(5884), tensor(0), tensor(2015), tensor(0), tensor(0), tensor(5731), tensor(0), tensor(2015), tensor(0), tensor(0), tensor(7678), tensor(3239), tensor(0), tensor(2015), tensor(0), tensor(0), tensor(7678), tensor(3239), tensor(0), tensor(2015), tensor(0), tensor(0), tensor(2758), tensor(5922), tensor(0), tensor(2015), tensor(0), tensor(0), tensor(5608), tensor(5888), tensor(2495), tensor(0), tensor(2015), tensor(0), tensor(0), tensor(7822), tensor(2103), tensor(0), tensor(2015), tensor(0), tensor(0), tensor(2788), tensor(3111), tensor(0), tensor(2038), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), ten

In [20]:
test_data['numerical_tokens'] = test_data['cleaned_tokens'].apply(lambda tokens: [vocab.get(token, 0) for token in tokens])

# Pad sequences using PyTorch
padded_sequences_test = pad_sequence([torch.tensor(tokens) for tokens in test_data['numerical_tokens']], batch_first=True, padding_value=0)
test_data['padded_sequences'] = [list(seq) for seq in padded_sequences_test]

In [22]:
# Define the LSTM model
class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, bidirectional, dropout):
        super(SentimentLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        lstm_output, _ = self.lstm(embedded)
        final_output = self.fc(lstm_output[:, -1, :])
        return final_output


# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(padded_sequences, train_data['label'].values, test_size=0.2, random_state=1123)

# Convert data to PyTorch tensors and create DataLoader
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
val_dataset = torch.utils.data.TensorDataset(torch.tensor(X_val), torch.tensor(y_val))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Define hyperparameters
vocab_size = len(vocab) + 1  # Additional 1 for the padding token
embedding_dim = 100
hidden_dim = 128
output_dim = 1  # Binary classification (positive or negative sentiment)
num_layers = 2
bidirectional = True
dropout = 0.5

# Instantiate the LSTM model
lstm_model = SentimentLSTM(vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, bidirectional, dropout)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)

# Training loop
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lstm_model.to(device)

for epoch in range(num_epochs):
    lstm_model.train()
    for batch_data, batch_labels in train_loader:
        batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)

        # Forward pass
        outputs = lstm_model(batch_data)
        loss = criterion(outputs.squeeze(), batch_labels.float())

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation
    lstm_model.eval()
    val_predictions = []
    val_true_labels = []
    with torch.no_grad():
        for val_batch_data, val_batch_labels in val_loader:
            val_batch_data, val_batch_labels = val_batch_data.to(device), val_batch_labels.to(device)

            val_outputs = lstm_model(val_batch_data)
            val_predictions.extend(torch.round(torch.sigmoid(val_outputs)).cpu().numpy())
            val_true_labels.extend(val_batch_labels.cpu().numpy())

    val_accuracy = accuracy_score(val_true_labels, val_predictions)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Validation Accuracy: {val_accuracy}")


<ipython-input-22-a7836882be76>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
<ipython-input-22-a7836882be76>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = torch.utils.data.TensorDataset(torch.tensor(X_val), torch.tensor(y_val))


Epoch 1/5, Loss: 0.4181031882762909, Validation Accuracy: 0.7937840879577526
Epoch 2/5, Loss: 0.48933476209640503, Validation Accuracy: 0.7937840879577526
Epoch 3/5, Loss: 0.5084962248802185, Validation Accuracy: 0.7937840879577526
Epoch 4/5, Loss: 0.656769335269928, Validation Accuracy: 0.7937840879577526
Epoch 5/5, Loss: 0.41489875316619873, Validation Accuracy: 0.7937840879577526


In [23]:
# Create DataLoader for test data
test_dataset = torch.utils.data.TensorDataset(torch.tensor(padded_sequences_test), torch.tensor(test_data['label'].values))
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

lstm_model.eval()
test_predictions = []
test_true_labels = []

with torch.no_grad():
    for test_batch_data, test_batch_labels in test_loader:
        test_batch_data, test_batch_labels = test_batch_data.to(device), test_batch_labels.to(device)

        test_outputs = lstm_model(test_batch_data)
        test_predictions.extend(torch.round(torch.sigmoid(test_outputs)).cpu().numpy())
        test_true_labels.extend(test_batch_labels.cpu().numpy())

test_accuracy = accuracy_score(test_true_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy}")


<ipython-input-23-fcaa35ef06a3>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_dataset = torch.utils.data.TensorDataset(torch.tensor(padded_sequences_test), torch.tensor(test_data['label'].values))


Test Accuracy: 0.8173192771084338


# XAI Analysis with Captum

In [25]:
import random